In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Improving-critical-exponents_pytorch/')
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [ ]:
args_cnn = {
    'save_dir': './saved_models/cnn_regression',
    'lattice_size': 128,
    'dataset_size': 5000,
    'epochs': 300,
    'batch_size': 256,
    'dropout_rate': 0.0,
    'learning_rate': 10e-4,
    'device' : device,
    'save_checkpoints': False,
}
        
args_cnn = objectview(args_cnn)

In [ ]:
from src.cnn.train import main as main_cnn
main_cnn(args_cnn)

Epoch: 1/300, Train Loss: 0.0734, Test Loss: 0.0669, Time: 17.19s
Epoch: 2/300, Train Loss: 0.0285, Test Loss: 0.1283, Time: 17.14s


KeyboardInterrupt: ignored

In [ ]:
args_gen = {
    'save_dir': './saved_models/gan_cnn_regression',
    'epochs': 10,
    'batch_size': 5,
    'learning_rate': 10e-3,
    'device' : device,
    'bins_number': 100,
    'noise_dim': 100,
    'ckpt_freq': 10,
    'CNN_model_path': './saved_models/cnn_regression/2021.10.19.07.21.08/model/final_model.pt',
    'set_generate_plots': False,
    'regularization_strength': 0.5,
}

args_gen = objectview(args_gen)

In [ ]:
from src.generator.train import main as main_gen
main_gen(args_gen)

    - Episode: 0             | Loss: 0.00          | Duration in seconds: 0.07         
    - Episode: 1             | Loss: 0.35          | Duration in seconds: 0.04         
    - Episode: 2             | Loss: 0.35          | Duration in seconds: 0.03         
    - Episode: 3             | Loss: 0.35          | Duration in seconds: 0.03         
    - Episode: 4             | Loss: 0.35          | Duration in seconds: 0.03         
    - Episode: 5             | Loss: 0.35          | Duration in seconds: 0.03         
    - Episode: 6             | Loss: 0.35          | Duration in seconds: 0.03         
    - Episode: 7             | Loss: 0.35          | Duration in seconds: 0.03         
    - Episode: 8             | Loss: 0.35          | Duration in seconds: 0.04         
    - Episode: 9             | Loss: 0.35          | Duration in seconds: 0.66         
